In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 9.6 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [9]:

print(test_dataset.to_pandas().head(15))

print(test_dataset[:15])  # View the first 5 rows as a dictionary


                                                 text  label
0   lovingly photographed in the manner of a golde...      1
1               consistently clever and suspenseful .      1
2   it's like a " big chill " reunion of the baade...      1
3   the story gives ample opportunity for large-sc...      1
4                   red dragon " never cuts corners .      1
5   fresnadillo has something serious to say about...      1
6   throws in enough clever and unexpected twists ...      1
7   weighty and ponderous but every bit as filling...      1
8   a real audience-pleaser that will strike a cho...      1
9   generates an enormous feeling of empathy for i...      1
10  exposing the ways we fool ourselves is one hou...      1
11  it's up to you to decide whether to admire the...      1
12  mostly , [goldbacher] just lets her complicate...      1
13  . . . quite good at providing some good old fa...      1
14  at its worst , the movie is pretty diverting ;...      1
{'text': ['lovingly phot

# Google Word2Vec

In [19]:
# from gensim.models import KeyedVectors

# # Path to the .bin file
# model_path = 'GoogleNews-vectors-negative300.bin'  # Update with the exact path to the .bin file

# # Load the Word2Vec model
# word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
# print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
import os
import gensim.downloader as api
from gensim.models import KeyedVectors

# Define the model path
model_name = "word2vec-google-news-300"
model_path = f"{model_name}.bin"

# Check if the model file already exists
if not os.path.exists(model_path):
    # Download the model
    path = api.load(model_name, return_path=True)
    print(f"Model downloaded and saved at: {path}")
else:
    print(f"Model already exists at: {model_path}")

# Load the model
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
print("Model loaded successfully.")


In [23]:
# Find words similar to "king"
similar_words = word2vec_model.most_similar("king", topn=10)
print("Words similar to 'king':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

Words similar to 'king':
kings: 0.7138
queen: 0.6511
monarch: 0.6413
crown_prince: 0.6204
prince: 0.6160
sultan: 0.5865
ruler: 0.5798
princes: 0.5647
Prince_Paras: 0.5433
throne: 0.5422


In [25]:
# Compute similarity between "king" and "queen"
similarity = word2vec_model.similarity("king", "queen")
print(f"Similarity between 'king' and 'queen': {similarity:.4f}")

Similarity between 'king' and 'queen': 0.6511


In [27]:
# Get the vector for the word "apple"
vector = word2vec_model["apple"]
print("Vector for 'apple':", vector)

Vector for 'apple': [-0.06445312 -0.16015625 -0.01208496  0.13476562 -0.22949219  0.16210938
  0.3046875  -0.1796875  -0.12109375  0.25390625 -0.01428223 -0.06396484
 -0.08056641 -0.05688477 -0.19628906  0.2890625  -0.05151367  0.14257812
 -0.10498047 -0.04736328 -0.34765625  0.35742188  0.265625    0.00188446
 -0.01586914  0.00195312 -0.35546875  0.22167969  0.05761719  0.15917969
  0.08691406 -0.0267334  -0.04785156  0.23925781 -0.05981445  0.0378418
  0.17382812 -0.41796875  0.2890625   0.32617188  0.02429199 -0.01647949
 -0.06494141 -0.08886719  0.07666016 -0.15136719  0.05249023 -0.04199219
 -0.05419922  0.00108337 -0.20117188  0.12304688  0.09228516  0.10449219
 -0.00408936 -0.04199219  0.01409912 -0.02111816 -0.13476562 -0.24316406
  0.16015625 -0.06689453 -0.08984375 -0.07177734 -0.00595093 -0.00482178
 -0.00089264 -0.30664062 -0.0625      0.07958984 -0.00909424 -0.04492188
  0.09960938 -0.33398438 -0.3984375   0.05541992 -0.06689453 -0.04467773
  0.11767578 -0.13964844 -0.2636

# GLOVE Embedding Model

In [38]:
import pickle

# Path to your .pickle file
glove_path = 'glove-wiki-gigaword-300.pickle'  # Replace with the actual file path

# Load the GloVe model from the pickle file
with open(glove_path, 'rb') as file:
    glove_model = pickle.load(file)

print("GloVe model loaded successfully.")

GloVe model loaded successfully.


In [40]:
# Step 2: Testing the GloVe model

# 1. Check similar words
print("Words similar to 'king':")
similar_words = glove_model.most_similar("king", topn=10)
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

# 2. Compute word similarity
similarity = glove_model.similarity("king", "queen")
print(f"\nSimilarity between 'king' and 'queen': {similarity:.4f}")

# 3. Perform analogy
analogy_result = glove_model.most_similar(positive=["woman", "king"], negative=["man"], topn=1)
print("\nAnalogy result for 'king - man + woman':", analogy_result)

# 4. Check for a word in the vocabulary
word = "apple"
if word in glove_model:
    print(f"\n'{word}' is in the vocabulary.")
else:
    print(f"\n'{word}' is not in the vocabulary.")

# 5. Get word vector
vector = glove_model["apple"]
print("\nVector for 'apple':", vector)

Words similar to 'king':
queen: 0.6336
prince: 0.6197
monarch: 0.5900
kingdom: 0.5791
throne: 0.5606
ii: 0.5562
iii: 0.5503
crown: 0.5225
reign: 0.5217
kings: 0.5066

Similarity between 'king' and 'queen': 0.6336

Analogy result for 'king - man + woman': [('queen', 0.6713277101516724)]

'apple' is in the vocabulary.

Vector for 'apple': [-0.20842   -0.019668   0.063981  -0.71403   -0.21181   -0.59283
 -0.15316    0.044217   0.63289   -0.84821   -0.21129   -0.19763
  0.19029   -0.56226    0.27126    0.23782   -0.5189    -0.24518
  0.035243   0.096833   0.24898    0.71279    0.038279  -0.10514
 -0.4779    -0.39515   -0.27194   -0.44428    0.06113   -0.2318
 -0.35901   -0.18239    0.035507  -0.087719  -1.0816    -0.42521
  0.003224  -0.45991   -0.043462  -0.39031    0.519      0.21139
 -0.25527    1.1805    -0.19041   -0.12156    0.034186  -0.062316
  0.14421   -0.53366    0.47425   -0.4471     0.58047    0.43578
  0.1321    -0.095712  -0.37182   -0.013837   0.20601   -0.10099
  0.10685  

# Q1a) What is the size of the vocabulary formed from your training data?

In [43]:
import re
from collections import Counter

def tokenize(text):
    # Use simple whitespace tokenization and lowercasing
    return re.findall(r'\b\w+\b', text.lower())

# Count unique tokens
counter = Counter()
for example in train_dataset:
    tokens = tokenize(example["text"])
    counter.update(tokens)

# Size of the vocabulary
vocab_size = len(counter)
print("Vocabulary size:", vocab_size)

Vocabulary size: 16512


# Q1b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [50]:
counter = 0
for example in train_dataset:
    tokens = tokenize(example["text"])
    tokens = [token.lower() for token in tokens]
    for token in tokens:
        if token not in glove_model:
            counter += 1
            print(token)

print("Number of OOV words:", counter)

wisegirls
enrapturing
compleja
intelectualmente
retadora
orquídeas
originalidad
suspenser
obviation
gorefests
waydowntown
makmalbaf
exhilarate
nuttgens
petin
provocatuers
jirí
hubac
shapelessly
addessi
seldahl
wollter
mullinski
avventura
needn
narcotizing
precollegiate
sparklingly
superlarge
destinees
margolo
dominatrixes
scuzbag
idoosyncratic
flatula
denlopp
updatings
watstein
sappier
condensada
divertida
visualmente
entretenida
sorprenderá
exporing
capturou
sarcástica
divertida
demencial
predecesora
complejos
cadness
shagster
powaqqatsi
policiales
últimos
kaputschnik
kickass
travil
splittingly
aborbing
monkeyfun
bierbichler
crummles
bustingly
stultifyingly
deutchland
datedness
inhospitability
næs
hastier
estava
existência
papai
fato
inquestionável
talancón
drippiness
seldahl
oesn
montias
hotdogging
stumblings
birot
apesar
consegue
entreter
recoing
bizzarre
wollter
seldahl
alientation
sogginess
birot
mollà
involvingly
wifty
gerbosi
stuffiest
timewaster
strafings
debuter
histo
contando

# Q1c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.